# ClearScape Analytics Experience Support Macros

These are used by associates developing and tuning notebooks, so they are not listed in the index of demos.

Those are:
 
- **recent** - get the list of sessions in the past hours, parameter is a number and defaults to 8
- **qrylog**  - get the queries executed in a session which must be passed as a parameter. If session completed within the last 10 minutes, issue:<br> **flush query logging with all**    
<br>
- **sessions** - get the sessionid and vproc of sessions logged on and in flight along with execution statistics
- **querytext** - for a sessionid and vproc, show the query running                                                                                                 
- **steps** - for a sessionid and vproc, show the steps executed and planned for a query in flight                                                       

In [ ]:
%connect local, hidewarnings=true

In [ ]:
replace macro recent(hours integer default 48) as (  -- identify sessionid that were recently executed - default 48 hours
    select sessionid
    , queryband, min(starttime) starting
    , max(firstresptime) ending
    , count(*)
    , sum(cast((firstresptime - starttime second(4)) as decimal(8,2))) RunSecs 
    from dbc.qrylog 
    where starttime > current_timestamp - interval '1' hour  * :hours
    group by 1,2 order by 1,2;
);

In [ ]:
REPLACE macro qrylog (sess integer default session) as ( -- pull all log records for 1 sessionid
    select starttime
    , firstresptime - starttime second (4) runsecs
    , '"' || otranslate(otranslate(querytext,'"',''),chr(10),' ') || '"' querytext
    , queryband, sessionid, ampcputime
    , 1-ampcputime/(nullifzero(MaxAMPCPUTime)*NumOfActiveAMPs) cpuskew
    , totaliocount
    , NosPhysReadIO
    , queryid from dbc.dbqlogtbl 
    where sessionid = :sess order by 1;
);

In [ ]:
REPLACE  macro sessions as ( -- list out all active sessions
    SELECT * FROM TABLE ( syslib.monitorsession(-1,'*',0)) AS x 
        ORDER BY pestate, username;
    );

In [ ]:
replace macro querytext (sess integer default session, vproc smallint default 30719) as ( -- get query text for a query now in flight
    select * from table (syslib.MONITORSQLTEXT (1,:sess,:vproc)) as x 
        order by 3 ;
    );

In [ ]:
replace MACRO steps(sess INTEGER default session, vproc SMALLINT default 30719) AS ( -- get steps from queryplan currently executing
    SELECT * FROM TABLE ( syslib.monitorsqlsteps(1,:sess,:vproc)) AS x 
        ORDER BY 3; 
    );